<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/main/Flan_T5_Base_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

Cloning into 'qM-AI-L'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 193 (delta 91), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (193/193), 7.98 MiB | 3.39 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [2]:
!pip install transformers
!pip install evaluate
!pip install rouge-score # Installing rouge-score library (https://pypi.org/project/rouge-score/)

# !pip install accelerate -U

!pip install transformers[torch]
!pip install sacrebleu

!pip install datasets nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Fou

In [3]:
# Importing Libraries

import os
import re
import pandas as pd # Data Handling
import tensorflow as tf
import numpy as np
import torch
import time
import seaborn as sns

from datasets import load_metric
from datasets import Dataset
import evaluate

from google.colab import files
import shutil

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
!pip install nltk
import nltk
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score

# **Load Dataset**

In [6]:
dataset_df = pd.read_csv('/content/qM-AI-L/email-subject/model-tuning/train_emails.csv')

In [7]:
dataset_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
0,allen-p_inbox_20,"Greg/Phillip, Attached is the Grande Communic...",Service Agreement,NaN,NaN,NaN,65,2,gregphillip attached is the grande communicati...
1,allen-p_inbox_28,Phillip & Keith Attached is the first draw re...,Bishops Corner,NaN,NaN,NaN,145,2,phillip keith attached is the first draw reque...
2,allen-p_inbox_63,Your Internet Banking accounts are now setup a...,Internet Banking,NaN,NaN,NaN,250,2,your internet banking accounts are now setup a...
3,allen-p_inbox_64,To our IBS Customers that are still hanging in...,Internet Banking,NaN,NaN,NaN,458,2,to our ibs customers that are still hanging in...
4,allen-p_inbox_65,Phillip Good Morning!\nI hope you had a wonder...,SMEs for expert stories,NaN,NaN,NaN,68,4,phillip good morning i hope you had a wonderfu...
...,...,...,...,...,...,...,...,...,...
14431,zufferli-j_inbox_43,This email is acknowledgement from the Power P...,Power Pool,NaN,NaN,NaN,227,2,this email is acknowledgement from the power p...
14432,zufferli-j_inbox_44,This email is acknowledgement from the Power P...,Power Pool of Alberta,NaN,NaN,NaN,277,4,this email is acknowledgement from the power p...
14433,zufferli-j_inbox_46,"John, Further to the voice message that I lef...",Enron Security,NaN,NaN,NaN,148,2,john further to the voice message that i left ...
14434,zufferli-j_inbox_8,Make sure that all curves are downloaded by th...,Simulation Curves,NaN,NaN,NaN,66,2,make sure that all curves are downloaded by th...


# **Test Train Split**

In [18]:
subset_df = dataset_df.sample(frac=0.3, random_state=42)
subset_df.shape

(4331, 9)

In [19]:
Train_df, Test_df = train_test_split(subset_df, test_size=0.2, random_state=42)

In [20]:
Train_df.shape, Test_df.shape

((3464, 9), (867, 9))

In [21]:
Train_dataset = Dataset.from_pandas(Train_df)
Test_dataset = Dataset.from_pandas(Test_df)

In [22]:
Train_dataset, Test_dataset

(Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__'],
     num_rows: 3464
 }),
 Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__'],
     num_rows: 867
 }))

**Create a subset of Dataset**

In [23]:
Train_dataset.shape, Test_dataset.shape

((3464, 10), (867, 10))

In [24]:
Train_dataset['cleaned_emails'][10]

'what do martinis jazz and george washington have in common fine arts friday a happy hour for professionals presented by the museum of fine arts houston conoco krts 92.1 friday march 1 2002 in the cafe express of the museums audrey jones beck building 5601 main street free admission 600 830 p.m. 5 martini specials complimentary hors doeuvres cash bar come as you are mix and mingle with other professionals and see the monumental lansdowne portrait of george washington by american painter gilbert stuart. now on view at the mfah the portrait made news in 2001 when the owner who had lent the painting to the national portrait gallery at the smithsonian institution since 1968 decided to sell it. the donald w. reynolds foundation of las vegas donated 30 million to allow the national portrait gallery to purchase the painting. the mfah is the first stop on the national tour george washington a national treasure.'

**Load the pre-trained FLAN-T5 models and its tokenizer directly from HuggingFace. FLAN-T5 was released in the paper Scaling Instruction-Finetuned Language Models - it is an enhanced version of T5 that has been finetuned in a mixture of tasks. Setting torch_dtype=torch.bfloat16 specifies the memory type to be used by this model.**

In [25]:
# Function to get size of models
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Trainable model parameters: {trainable_model_params}\nAll model parameters: {all_model_params}\nPercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



**Load Flan T5 Base Model**

In [26]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


print("For Model : FLAN-T5-Base \n",print_number_of_trainable_model_parameters(model))

For Model : FLAN-T5-Base 
 Trainable model parameters: 247577856
All model parameters: 247577856
Percentage of trainable model parameters: 100.00%


In [27]:
def preprocess_function(examples):
    inputs = [f"Generate a subject line for the following email: {email}" for email in examples['cleaned_emails']]
    targets = examples['subject']
    model_inputs = tokenizer(inputs, max_length=512, add_special_tokens=True, truncation=True, padding='max_length')

    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the dataset
tokenized_train_dataset = Train_dataset.map(preprocess_function, batched=True, remove_columns=Train_dataset.column_names)
tokenized_eval_dataset = Test_dataset.map(preprocess_function, batched=True, remove_columns=Test_dataset.column_names)

Map:   0%|          | 0/3464 [00:00<?, ? examples/s]

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

In [28]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3464
})

# **Test the Model with Zero Shot Inferencing**

**Using a pre-trained model from Hugging Face/sources - FLAN-T5-Base .**

In [29]:
index = 9

email = Train_dataset['body'][index]
subject = Train_dataset['subject'][index]

prompt = f"""
Generate a subject line for the following email.

Email:
{email}

Subject:
"""

inputs = tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(model.device) for k, v in inputs.items()}


output = tokenizer.decode(model.generate(inputs["input_ids"], max_new_tokens=200)[0],skip_special_tokens=True)


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FLAN-T5 MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Generate a subject line for the following email.

Email:
We have received the following financial Master Agreement:  Type of Contract:		ISDA Master Agreement (Multicurrency-Cross Border)  Effective Date:			March 9, 2001  Enron Entity:			Enron North America Corp.  Counterparty:			Catequil Partners, L.P.
Transactions Covered:		Approved for all products with the exception of:  Foreign Exchange  Confirming Entity:		Enron North America Corp.  Governing Law:			New York  Special Handling Requirement:	Counterparty has five (5) Local Business Days after a confirmation was effectively received by it to accept or dispute a confirmation.
Copies will be distributed.

Subject:

---------------------------------------------------------------------------------------------------
BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :
Catequil Partners, L.P.

----------------------------------------

**By testing with various models with the zero shot inferencing, we can see that the model struggles to extract the same subject line compared to the baseline subject, but it does pull out some important information from the email which indicates the models can be fine-tuned to the task at hand.**

# **Performing Full Fine-Tuning using FLAN-T5 BASE MODEL**

In [30]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=3,
    logging_dir='./logs',
    logging_steps=500,
)

In [31]:
# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,30.246200,25.399366


TrainOutput(global_step=866, training_loss=28.435623556581987, metrics={'train_runtime': 791.1616, 'train_samples_per_second': 4.378, 'train_steps_per_second': 1.095, 'total_flos': 2371999468879872.0, 'train_loss': 28.435623556581987, 'epoch': 1.0})

In [34]:
# Save the model and tokenizer
save_directory = "./finetuned_flan_t5_base"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('./finetuned_flan_t5_base/tokenizer_config.json',
 './finetuned_flan_t5_base/special_tokens_map.json',
 './finetuned_flan_t5_base/spiece.model',
 './finetuned_flan_t5_base/added_tokens.json',
 './finetuned_flan_t5_base/tokenizer.json')

In [35]:
# Zip the saved model files
zip_filename = "finetuned_flan_t5_base.zip"
shutil.make_archive(base_name="finetuned_flan_t5_base", format="zip", root_dir=save_directory)


'/content/finetuned_flan_t5_base.zip'

In [36]:
#Download the zipped checkpoint
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [73]:
# Path to the model directory in Google Drive
model_path = '/content/drive/MyDrive/Colab Notebooks/finetuned_flan_t5_base11'

In [74]:
# Load the fine-tuned model and tokenizer

finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
finetuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [75]:
val_dataset_df = pd.read_csv('/content/qM-AI-L/email-subject/model-tuning/test_emails.csv')

In [76]:
val_dataset_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
0,allen-p_inbox_24,"Phillip, Could you please do me a favor?\nI ...",Huntley/question,"phillip, could you please do me a favor?",need to review your title policy's easements s...,need copy of title policy,70,1,phillip could you please do me a favor i would...
1,allen-p_inbox_45,The following reports have been waiting for yo...,Expense Reports Awaiting Your Approval,action needed re: reitmeyer report,report review reminder,still waiting for your report approvals,29,5,the following reports have been waiting for yo...
2,allen-p_inbox_6,We have for the last couple of weeks started t...,Re-start/Integration Planning,"time sensitive, re-start/integration plans needed",new focus: commercial plans,status of netco re-start/integration plans,263,2,we have for the last couple of weeks started t...
3,allen-p_inbox_66,Michelle Here are my very minor comments.\nHow...,RM Simulation Storyline Scripts - Ready for Le...,cnn type theme issues?,minor comments- promotions,some thoughts subsequent to sme meeting,164,9,michelle here are my very minor comments. howe...
4,allen-p_inbox_84,For purposes of an accelerated distribution fr...,Answer,psa decision and explanation,information regarding a psa account.,clarification regarding psa and single sum dis...,206,1,for purposes of an accelerated distribution fr...
...,...,...,...,...,...,...,...,...,...
1901,zipper-a_inbox_39,"Andy, Stephanie Sever has contacted ICE and h...",ICE,access list update,ice forms and access,ice access,40,1,andy stephanie sever has contacted ice and has...
1902,zipper-a_inbox_62,The following was developed in an effort to as...,Non-Exempt Scorecard,assistance for evaluation of non-exempt employees,assistance with evaluating non-exempt employees,non-exempt evaluation document,26,2,the following was developed in an effort to as...
1903,zipper-a_inbox_90,REMINDER: Your weekly update for Philippe and...,Weekly Reports are Due,weekly update - phil & greg,weekly update reminder,weekly reports due,26,4,reminder your weekly update for philippe and g...
1904,zufferli-j_inbox_1,"Please find attached the latest, and what shou...",Latest Marketing List,marketing list assignments,marketing list,marketing list attached,183,3,please find attached the latest and what shoul...


In [80]:
emails_for_scores_df = val_dataset_df.sample(frac=0.05, random_state=22)
emails_for_scores_df.shape

(95, 9)

In [81]:
emails_for_scores_df.dtypes

source            object
body              object
subject           object
ann0              object
ann1              object
ann2              object
body_wcount        int64
subj_wcount        int64
cleaned_emails    object
dtype: object

In [82]:
original_subjects = []
flan_T5_subjects = []
finetuned_flan_T5_subjects = []

ann0  = []
ann1  = []
ann2  = []



for index, row in emails_for_scores_df.iterrows():
    email = row['cleaned_emails']
    original_subject = row['subject']
    ann00= row['ann0']
    ann11= row['ann1']
    ann22= row['ann2']
    prompt = f"""
    Generate a subject line for the following email.

    Email:
    {email}

    Subject:
    """

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    finetuned_model.to(device)

    original_model_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    finetuned_model_input_ids = finetuned_tokenizer(prompt, return_tensors="pt").input_ids.to(device)


    original_model_outputs = model.generate(input_ids=finetuned_model_input_ids, num_beams=4,generation_config=GenerationConfig(max_new_tokens=100))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], clean_up_tokenization_spaces=True, skip_special_tokens=True)
    flan_T5_subjects.append(original_model_text_output)

    instruct_model_outputs = finetuned_model.generate(input_ids=original_model_input_ids, max_length=512,
      repetition_penalty=2.5,
      length_penalty=1,
      early_stopping=True,
      top_p=.95,
      top_k=50,
      num_return_sequences=3,
      num_beams=4,
      generation_config=GenerationConfig(max_new_tokens=100))
    instruct_model_text_output = finetuned_tokenizer.decode(instruct_model_outputs[0], clean_up_tokenization_spaces=True, skip_special_tokens=True)
    finetuned_flan_T5_subjects.append(instruct_model_text_output)

    original_subjects.append(original_subject)
    ann0.append(ann00)
    ann1.append(ann11)
    ann2.append(ann22)

zipped_subjects = list(zip(original_subjects, flan_T5_subjects, finetuned_flan_T5_subjects, ann0, ann1, ann2))

Flan_T5_df = pd.DataFrame(zipped_subjects, columns = ['True_subjects', 'Flan_T5_subjects', 'Finetuned_flan_T5_subjects', 'ann0', 'ann1', 'ann2'])
Flan_T5_df

Both `max_new_tokens` (=100) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


,True_subjects,Flan_T5_subjects,Finetuned_flan_T5_subjects,ann0,ann1,ann2
0,CALIFORNIA UPDATE - 9/4/2001,edison mou - a snapshot analysis of the politi...,Executive Summary of the Edison Mou,executive summary,executive summary: edison mou,update on appropriations committee's plan
1,PLEASE READ - IMPORTANT INFORMATION FOR PARTIC...,Enron Corp. Savings Plan Announcement,Enron Savings Plan,please note this amendment to the enron saving...,important changes made to enron corp. savings ...,important amendments to enron corp. savings plan
2,Franky Sulistio,franky sulistio,Gas Fundamentals It,fran sulistio introduction,new addition to gas fundamentals it group: fra...,franky sulistio joining gas fundamentals it group
3,Turlock Irrigation District,Turlock Master Firm Purchase Sale Agreement,Turlock Master Sale Agreement,here's the turlock agreement draft to distribute,turlock firm purchase/sale agreement instructions,master firm purchase/ sale agreement draft
4,NDA-Sabre Corporation,David Endicotts Non-Disclosure Agreement for D...,Nondisclosure Agreement,david endicott's review needed for non-disclos...,proposed non-disclosure agreement for review,comments please: nda draft + changes
...,...,...,...,...,...,...
90,WSCC-S Index Option,wsccs - wsccs is your financial position w the...,Financial Position w the index option,financial position,your financial position with the index option,your position/strike/deal type
91,EOL Summary - Still Under Construction,eol report,EOL Status Report,update on it work in houston,attempt to restore eol report,efforts to recapture that ol' eol report magic...
92,EOL - Multiple Trading Books (URGENT),eol trading book issue,EOL and Multiple Trading Books,multiple trading books,multiple trading book problem,eol problem with multiple trading books
93,A.Word.A.Day--cecity,oddsandends.com - oddsandends.com - oddsandend...,EOL and Clickpaper Approvals,one-of-a-kind words,the gift of your word of the day,cecity defind


In [83]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

finetuned_model_results = rouge.compute(
    predictions=finetuned_flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

ann0_original_model_results = rouge.compute(
    predictions=flan_T5_subjects,
    references=ann0,
    use_aggregator=True,
    use_stemmer=True,
)

ann0_finetuned_model_results = rouge.compute(
    predictions=finetuned_flan_T5_subjects,
    references=ann0,
    use_aggregator=True,
    use_stemmer=True,
)

ann1_original_model_results = rouge.compute(
    predictions=flan_T5_subjects,
    references=ann1,
    use_aggregator=True,
    use_stemmer=True,
)

ann1_finetuned_model_results = rouge.compute(
    predictions=finetuned_flan_T5_subjects,
    references=ann1,
    use_aggregator=True,
    use_stemmer=True,
)


ann2_original_model_results = rouge.compute(
    predictions=flan_T5_subjects,
    references=ann2,
    use_aggregator=True,
    use_stemmer=True,
)

ann2_finetuned_model_results = rouge.compute(
    predictions=finetuned_flan_T5_subjects,
    references=ann2,
    use_aggregator=True,
    use_stemmer=True,
)


zipped_results = list(zip(original_model_results, finetuned_model_results,
                          ann0_original_model_results, ann0_finetuned_model_results,
                          ann1_original_model_results, ann1_finetuned_model_results,
                          ann2_original_model_results, ann2_finetuned_model_results))

print('FLAN T5 Base Model Vs. Original Subjects:')
print(original_model_results)
print('\nFINETUNED FLAN T5 BASE MODEL Vs. Original Subjects:')
print(finetuned_model_results)

print('\nFLAN T5 Base Model Vs. ann0:')
print(ann0_original_model_results)
print('\nFINETUNED FLAN T5 BASE MODEL Vs. ann0:')
print(ann0_finetuned_model_results)

print('\nFLAN T5 Base Model Vs. ann1:')
print(ann1_original_model_results)
print('\nFINETUNED FLAN T5 BASE MODEL Vs. ann1:')
print(ann1_finetuned_model_results)

print('\nFLAN T5 Base Model Vs. ann1:')
print(ann2_original_model_results)
print('\nFINETUNED FLAN T5 BASE MODEL Vs. ann1:')
print(ann2_finetuned_model_results)
print('\n')


FLAN T5 Base Model Vs. Original Subjects:
{'rouge1': 0.23839251167102787, 'rouge2': 0.1208889518496201, 'rougeL': 0.2348413848322875, 'rougeLsum': 0.23276847909347015}

FINETUNED FLAN T5 BASE MODEL Vs. Original Subjects:
{'rouge1': 0.2658988964252121, 'rouge2': 0.10601921470342524, 'rougeL': 0.25889178073388586, 'rougeLsum': 0.26033548323022}

FLAN T5 Base Model Vs. ann0:
{'rouge1': 0.26587088159113403, 'rouge2': 0.14536800731879002, 'rougeL': 0.2489167850678122, 'rougeLsum': 0.2490957538725301}

FINETUNED FLAN T5 BASE MODEL Vs. ann0:
{'rouge1': 0.29243549707945987, 'rouge2': 0.13866332497911452, 'rougeL': 0.27433362818811735, 'rougeLsum': 0.2755296939043069}

FLAN T5 Base Model Vs. ann1:
{'rouge1': 0.2720536090165073, 'rouge2': 0.1454962690670338, 'rougeL': 0.25151475495384046, 'rougeLsum': 0.2518963321329426}

FINETUNED FLAN T5 BASE MODEL Vs. ann1:
{'rouge1': 0.3071219423851002, 'rouge2': 0.19062200956937803, 'rougeL': 0.30053639343113026, 'rougeLsum': 0.301829457092615}

FLAN T5 Bas

In [84]:
from collections import defaultdict

# Initialize a defaultdict with zeroes for numeric values
result = defaultdict(int)

# List of dictionaries to concatenate
dicts = [original_model_results, finetuned_model_results,
         ann0_original_model_results, ann0_finetuned_model_results,
         ann1_original_model_results, ann1_finetuned_model_results,
         ann2_original_model_results, ann2_finetuned_model_results]

scores_df = pd.DataFrame(dicts)
new_index = ['True_subjects Vs. Base_model', 'True_subjects Vs. Finetuned_model',
              'Ann0 Vs. Base_model', 'Ann0 Vs. Finetuned_model',
             'Ann1 Vs. Base_model','Ann1 Vs. Finetuned_model',
              'Ann2 Vs. Base_model','Ann2 Vs. Finetuned_model']

scores_df.index = new_index
scores_df


,rouge1,rouge2,rougeL,rougeLsum
True_subjects Vs. Base_model,0.238393,0.120889,0.234841,0.232768
True_subjects Vs. Finetuned_model,0.265899,0.106019,0.258892,0.260335
Ann0 Vs. Base_model,0.265871,0.145368,0.248917,0.249096
Ann0 Vs. Finetuned_model,0.292435,0.138663,0.274334,0.275530
Ann1 Vs. Base_model,0.272054,0.145496,0.251515,0.251896
Ann1 Vs. Finetuned_model,0.307122,0.190622,0.300536,0.301829
Ann2 Vs. Base_model,0.250739,0.134479,0.240002,0.238884
Ann2 Vs. Finetuned_model,0.318985,0.185258,0.310836,0.310023


In [85]:
scores_df = scores_df.T
scores_df.T.columns

Index(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], dtype='object')

In [86]:
original_model_results

{'rouge1': 0.23839251167102787,
 'rouge2': 0.1208889518496201,
 'rougeL': 0.2348413848322875,
 'rougeLsum': 0.23276847909347015}

In [96]:

print("For Ture Subjects: Absolute percentage improvement of FINETUNED MODEL over PRETRAINED")

improvement = (np.array(list(finetuned_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(finetuned_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

For Ture Subjects: Absolute percentage improvement of FINETUNED MODEL over PRETRAINED
rouge1: 2.75%
rouge2: -1.49%
rougeL: 2.41%
rougeLsum: 2.76%


In [97]:

print("For Ann2 : Absolute percentage improvement of FINETUNED MODEL over PRETRAINED")

improvement = (np.array(list(ann2_finetuned_model_results.values())) - np.array(list(ann2_original_model_results.values())))
for key, value in zip(finetuned_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

For Ann2 : Absolute percentage improvement of FINETUNED MODEL over PRETRAINED
rouge1: 6.82%
rouge2: 5.08%
rougeL: 7.08%
rougeLsum: 7.11%


In [88]:
!pip install nltk

import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Download the 'punkt' resource
nltk.download('punkt')

nltk.download('wordnet') # Download the WordNet corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [89]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [90]:
# Initialize metric scorers
rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = rouge_scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([word_tokenize(true_subject)], word_tokenize(generated_subject))

    # SACREBLEU
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    # BLEU
    bleu_score = sentence_bleu([word_tokenize(true_subject)], word_tokenize(generated_subject))


    return rougeL, meteor, sacre_bleu_score, bleu_score

# Apply metrics evaluation
Flan_T5_df['rougeL'], Flan_T5_df['meteor'], Flan_T5_df['sacrebleu'], Flan_T5_df['bleu'] = zip(*Flan_T5_df.apply(
    lambda row: evaluate_metrics(row['True_subjects'], row['Flan_T5_subjects']), axis=1))

# Calculate average scores
average_rougeL = Flan_T5_df['rougeL'].mean()
average_meteor = Flan_T5_df['meteor'].mean()
average_sacrebleu = Flan_T5_df['sacrebleu'].mean()
average_bleu = Flan_T5_df['bleu'].mean()


print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')
print(f'Average BLEU: {average_bleu:.4f}')


Average ROUGE-L: 0.2344
Average METEOR: 0.2092
Average SACREBLEU: 2.3803
Average BLEU: 0.0031


In [91]:
# Initialize metric scorers
from rouge_score import rouge_scorer
rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = rouge_scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([word_tokenize(true_subject)], word_tokenize(generated_subject))

    # SACREBLEU
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    # BLEU
    bleu_score = sentence_bleu([word_tokenize(true_subject)], word_tokenize(generated_subject))


    return rougeL, meteor, sacre_bleu_score, bleu_score

# Apply metrics evaluation
Flan_T5_df['rougeL'], Flan_T5_df['meteor'], Flan_T5_df['sacrebleu'], Flan_T5_df['bleu'] = zip(*Flan_T5_df.apply(
    lambda row: evaluate_metrics(row['True_subjects'], row['Finetuned_flan_T5_subjects']), axis=1))

# Calculate average scores
average_rougeL = Flan_T5_df['rougeL'].mean()
average_meteor = Flan_T5_df['meteor'].mean()
average_sacrebleu = Flan_T5_df['sacrebleu'].mean()
average_bleu = Flan_T5_df['bleu'].mean()


print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')
print(f'Average BLEU: {average_bleu:.4f}')

Average ROUGE-L: 0.2608
Average METEOR: 0.1928
Average SACREBLEU: 5.1273
Average BLEU: 0.0190


# **Perform Parameter Efficient Fine-Tuning (PEFT)**
Parameter Efficient Fine-Tuning (PEFT), which is more efficient than full fine-tuning and yields comparable results. PEFT, often referring to Low-Rank Adaptation (LoRA), enables fine-tuning with fewer compute resources, often a single GPU.

LoRA produces a small adapter (a few MBs) while keeping the original LLM unchanged. During inference, this adapter is combined with the original LLM, allowing multiple adapters to reuse the same LLM and reducing memory requirements for various tasks.

## **Setup the PEFT/LoRA model for Fine-Tuning**



In [92]:
!pip install \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

In [93]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [94]:
peft_model = get_peft_model(model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

Trainable model parameters: 3538944
All model parameters: 251116800
Percentage of trainable model parameters: 1.41%
